In [31]:
# @hidden_cell
# The project token is an authorization token that is used to access project resources like data sources, connections, and used by platform APIs.
from project_lib import Project
project = Project(project_id='18c9dc37-6f05-4132-a583-46842caa6329', project_access_token='p-2b4eb6562dcd081943e8d03040e939cec1c79724')
pc = project.project_context


# A data-driven way to merge similar classes together

Juan Lopez Martin

The quality of the classes is determinant for training a classifier. In the famous MNIST example, the handwritten number '6' was always labeled correclty as a '6'. However, it is clear it would be very problematic if that digit was sometimes labels as '6', others as 'six', 'SIX', '6s', etc. In this case, it would be necessary to merge all these labels together before training. If not, the classifier would probably pick up irrelevant features of the '6' to try to distinguish between the classes '6', 'six', '6s', etc and end up with much less prediction accuracy.

We are in a similar situation if we want to train a classifier to infer the issue from the narrative. For instance, some narratives have the issue 'Incorrect information on credit report', while others have 'Incorrect information on your report'. It is relatively clear that these two issues are not different, and that these two labels should be merged. The same happens, for instance, with 'Attempts to collect debt not owed' and 'Cont'd attempts collect debt not owed'.

## 0. Loading data

In [3]:
# Install gensim
!pip install gensim

     |████████████████████████████████| 24.2MB 12.4MB/s eta 0:00:01
     |████████████████████████████████| 71kB 28.1MB/s eta 0:00:01
  Stored in directory: /home/dsxuser/.cache/pip/wheels/ab/10/93/5cff86f5b721d77edaecc29959b1c60d894be1f66d91407d28
Successfully built smart-open


In [4]:
# Import packages for IBM Studio to load data

import types
from botocore.client import Config
import ibm_boto3

In [5]:
# Import packages
import pandas as pd
import numpy as np
from datetime import datetime
import pickle
from operator import itemgetter
from itertools import compress
#import spacy
#import en_core_web_sm
#nlp = en_core_web_sm.load()
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import networkx 
from networkx.algorithms.components.connected import connected_components
from sklearn.metrics import pairwise_distances
from scipy.spatial.distance import cosine


In [6]:
# Load dataset

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_51cbd8ac4f124ba699737e1f03820b07 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='6FMtmurbcuYnZjqEkX-88omiJcLBrUuUuEmuWnVe5Erk',
    ibm_auth_endpoint="https://iam.ng.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_51cbd8ac4f124ba699737e1f03820b07.get_object(Bucket='qmssdseprojectfall2019-donotdelete-pr-ju1hanupbqko6m',Key='Consumer_Complaints.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df = pd.read_csv(body)


/opt/conda/envs/Python36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (5,6,11,16) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
# Load pre-processed texts

streaming_body_2 = client_51cbd8ac4f124ba699737e1f03820b07.get_object(Bucket='qmssdseprojectfall2019-donotdelete-pr-ju1hanupbqko6m', Key='texts.pickle')['Body']
texts = pickle.loads(streaming_body_2.read())

In [8]:
# Load pre-processed documents (same as texts with gensim format)

streaming_body_3 = client_51cbd8ac4f124ba699737e1f03820b07.get_object(Bucket='qmssdseprojectfall2019-donotdelete-pr-ju1hanupbqko6m', Key='documents.pickle')['Body']
documents = pickle.loads(streaming_body_3.read())

In [9]:
# Load trained doc2vec model

streaming_body_4 = client_51cbd8ac4f124ba699737e1f03820b07.get_object(Bucket='qmssdseprojectfall2019-donotdelete-pr-ju1hanupbqko6m', Key='model_100.pickle')['Body']
model = pickle.loads(streaming_body_4.read())

In [10]:
# Throw away rows without narrative

df = df.dropna(subset=['Consumer complaint narrative'])

In [11]:
#This is to get a subset of the dataset. p=1 means we get 100% of the rows, 
# so in this case we are getting all of it. It is useful for exploring new 
# approaches.

p = 1
rand = np.random.choice(a=[False, True], size=len(texts), p = [1-p, p])

df_s = df[rand]

texts_s = list(compress(texts, rand))
len(texts_s)

documents_s = list(compress(documents, rand))
len(documents_s)

444683

In [59]:
df_s.shape

(444683, 18)

In [12]:
# We use L2 normalization on the vectors
model.docvecs.init_sims(replace=True)

# Then take the appropiate sample (again, by default we get the entire dataset)
vecs_s = []
for i in range(0, df.shape[0]):
  if rand[i] == True:
    vecs_s.append(model.docvecs[i])
    
len(vecs_s)

444683

## 1. Data cleaning (skip, output pickled)

We re using spacy to:

* Remove the X, XX, XXX, etc.
* Remove strings that are not alphanumeric
* Remove spaces
* Remove punctuation
* Remove numbers
* Lemmatize

This trasnforms the narrative to a list of words.

In [14]:
## Commented out -- output saved at model_100.pickle
 
#xs = ["X"*2, "X"*3, "X"*4, "X"*5, "X"*6, "X"*7, "X"*8, "X"*9, "X"*10, "X"*11, "XX/XX", "XX/XX/XXX", "XX/XX/XX", "XX/XXXX"]

#texts = []
#for sent in nlp.pipe(df['Consumer complaint narrative'], disable=["tagger", "parser", "ner", "textcat"]):
#    texts.append([word.lemma_ for word in sent 
#                  if word.is_alpha and
#                  not word.is_space and
#                  not word.is_stop and 
#                  not word.is_punct and 
#                  not word.like_num and 
#                     word.text not in xs])
    
#filename = folder+'texts.pickle'
#outfile = open(filename,'wb')
#pickle.dump(texts,outfile)
#outfile.close()

## 2. Doc2vec (skip, output pickled)

Word embeddings are vector representations of individual words that carry their semantic meaning. If these word vectors are obtained from a large corpus we would expect, for instance, that the vector for frog and toad will be very similar. Additionally, these vectors capture more complex relationships. For instance, the association between 'Washington' and USA will be similar as the association between 'China' and 'Beijin'.

Doc2vec is an extension to the famous word embedding approach that trains vector representations of entire documents instead of just words. If trained on a large set of books we would expect, for instance, that Statistics books will have similar vectors.

In this case, we will train document embeddings based on the narratives. In other words, each narrative will be transformed into a 300-dimensional vector. The main advantge of this approach over simpler methods is that it takes into account the order of the words in sentences. In fact, under the hood the vectors are built by trying to predict a center word based on the surrounding words (PV-DM). This allows to capture complex semantic relationships between words.

The hyperparamethers used here are relatively standard in the field:

* 300-dimensional vectors.
* Window of 15 words (meaning we try to predict the center word from a 15-word window).
* Negative sampling of 5 -- improves results and reduces computational cost.
* 100 epochs. This is good but it took 4 hours.


In [15]:
## Commented out -- output saved at model_100.pickle

# First, we create a taggeddocument object, necessary for word2vec. Then we
# train the vecs.

#documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(texts)]
#model = Doc2Vec(documents, dm = 1, vector_size=300, window=15, negative=5, min_count=1, workers=4, epochs = 100)

#filename = folder+'model_100.pickle'
#outfile = open(filename,'wb')
#model = pickle.dump(model, outfile)
#outfile.close()

## 3. Labels to merge

For each issue (class) we have a set of narratives (documents) that have been represented as (300-dimensional L2-normalized) vectors with the doc2vec algorithm. For example, for issue $A$ ('Incorrect information on your report') we have obtained $a_1, a_2, ..., a_{65508}$ vectors. One basic approach to summarize the information we have about the issue $A$ is to calculate the mean for all the vectors $a_i$. Doing this, we obtain an average (300-dimensional L2-normalized) vector that can be seen as an approximate representation of the issue $A$. The idea is that the average of the vectors $a_i$ should contain that is to a certain degree common for the issue $A$.   

In [13]:
# Create list of issues ordered by value_counts
issues_list = list(df_s['Issue'].value_counts().index)

# Calculate the mean for the vectors corresponding to each issue
vec_list = [np.mean(np.array(list(compress(vecs_s, list(df[rand]['Issue']==issue)))), axis  = 0) for issue in issues_list]

In [23]:
# vecs_s is list of vectorized narratives for the whole dataset (len 444683)

444683

#### a. First Round of Clustering/issue reduction (using original issue labels)

Now we have a vector representation of issue $A$, $B$, $C$, etc. In word2vec the similarity of two vectors is measured using cosine similarity, which determines whether the two vectors are pointing in approximately the same direction.


We will create a similarity matrix that reports the cosine similarity between all the issues. If you have no experience with word2vec, this can be conceptually similar to a correlation matrix. That is, issues that are similar between each other will have a cosine similarity close to one and non-related issues a score close to zero.

In [15]:
# Create distance matrix with cosine similarity
dist = 1-pairwise_distances(np.array(vec_list), metric="cosine")

# Put the distance matrix in a dataframe with index and column names
dist_df = pd.DataFrame(dist, index = issues_list, columns=issues_list)

dist_df.iloc[0:5,0:5]

,Incorrect information on your report,Problem with a credit reporting company's investigation into an existing problem,Attempts to collect debt not owed,Incorrect information on credit report,Improper use of your report
Incorrect information on your report,1.000000,0.988587,0.960438,0.990585,0.962792
Problem with a credit reporting company's investigation into an existing problem,0.988587,1.000000,0.947416,0.988502,0.950098
Attempts to collect debt not owed,0.960438,0.947416,1.000000,0.947011,0.929568
Incorrect information on credit report,0.990585,0.988502,0.947011,1.000000,0.955511
Improper use of your report,0.962792,0.950098,0.929568,0.955511,1.000000


We have to set up a threshold to decide which issues to merge and which issues to keep separate. In this case I have decided for 0.985, and in my experience something between 0.98 and 0.99 seems like a reasonable threshold. However, we can discuss the appropiate threshold. 


Note that merges of  issues can be multiple. Namely, if issue $A$ is going to be merged with $B$ and $B$ is also going to be merged with $C$, we are going to obtain one new issue $K = A \cup B \cup C$. This is similar to say that, given a network $G = (V, E)$ in which each vertex is an original issue and the edges are $1$ if the issues are going to be merged and $0$ otherwise, we are going to keep all the distinct components as labels.

In [16]:
# The issues to merge are decided according to an user-defined threshold.
threshold = 0.985
l = [list(dist_df[dist_df.loc[issue]>threshold].index) for issue in issues_list]

# Finding components in the network.
# Code from https://stackoverflow.com/questions/4842613/merge-lists-that-share-common-elements?lq=1

def to_graph(l):
    G = networkx.Graph()
    for part in l:
        # each sublist is a bunch of nodes
        G.add_nodes_from(part)
        # it also imlies a number of edges:
        G.add_edges_from(to_edges(part))
    return G

def to_edges(l):
    """ 
        treat `l` as a Graph and returns it's edges 
        to_edges(['a','b','c','d']) -> [(a,b), (b,c),(c,d)]
    """
    it = iter(l)
    last = next(it)

    for current in it:
        yield last, current
        last = current    

G = to_graph(l)
components = list(connected_components(G))

for aset in components:
  if len(aset)>1:
    for element in aset:
      print(element)
    print("\n")

Credit reporting company's investigation
Incorrect information on your report
Incorrect information on credit report
Problem with a credit reporting company's investigation into an existing problem


Cont'd attempts collect debt not owed
Attempts to collect debt not owed


Trouble during payment process
Loan servicing, payments, escrow account


Disclosure verification of debt
Written notification about debt


Struggling to pay mortgage
Loan modification,collection,foreclosure


Deposits and withdrawals
Managing an account


Took or threatened to take negative or legal action
Taking/threatening an illegal action


Application, originator, mortgage broker
Applying for a mortgage or refinancing an existing mortgage


Can't repay my loan
Struggling to repay your loan


Unable to get your credit report or credit score
Unable to get credit report/credit score


Problems caused by my funds being low
Problem caused by your funds being low




In [17]:
df['cluster_1'] = np.nan

In [18]:
print(len(components)) # we've boiled down 166 issues to 148 as part of the first round of clustering
# now we want to give each of these clusters a number ID, and assign them where appropriate in the dataframe

for i, comp in enumerate(components):
    df['cluster_1'].loc[df['Issue'].isin(list(comp))] = i
    print(i, comp)

148


/opt/conda/envs/Python36/lib/python3.6/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


0 {"Credit reporting company's investigation", 'Incorrect information on your report', 'Incorrect information on credit report', "Problem with a credit reporting company's investigation into an existing problem"}
1 {"Cont'd attempts collect debt not owed", 'Attempts to collect debt not owed'}
2 {'Improper use of your report'}
3 {'Trouble during payment process', 'Loan servicing, payments, escrow account'}
4 {'Communication tactics'}
5 {'Disclosure verification of debt', 'Written notification about debt'}
6 {'Struggling to pay mortgage', 'Loan modification,collection,foreclosure'}
7 {'False statements or representation'}
8 {'Deposits and withdrawals', 'Managing an account'}
9 {'Dealing with your lender or servicer'}
10 {'Dealing with my lender or servicer'}
11 {'Managing the loan or lease'}
12 {'Took or threatened to take negative or legal action', 'Taking/threatening an illegal action'}
13 {'Account opening, closing, or management'}
14 {'Problem with a purchase shown on your statement'

#### b. Repeat steps twice // we could write a for loop that takes care of this for us but now stick to mechanical code

In [19]:
# Create list of issues ordered by value_counts
issues_list = list(df['cluster_1'].value_counts().index)

# Calculate the mean for the vectors corresponding to each issue
vec_list = [np.mean(np.array(list(compress(vecs_s, list(df[rand]['cluster_1']==issue)))), axis  = 0) for issue in issues_list]

In [20]:
dist = 1-pairwise_distances(np.array(vec_list), metric="cosine")

# Put the distance matrix in a dataframe with index and column names
dist_df = pd.DataFrame(dist, index = issues_list, columns=issues_list)

In [21]:
dist_df.iloc[0:5,0:5] # notice how similarities have dropped, this is normal because more different narratives are clustered together
# what should we pick for the new threshold? we'll pick 0.9 to start with

,0.0,1.0,3.0,5.0,2.0
0.0,1.000000,0.957306,0.854775,0.948433,0.961159
1.0,0.957306,1.000000,0.835960,0.983153,0.929901
3.0,0.854775,0.835960,1.000000,0.823749,0.848809
5.0,0.948433,0.983153,0.823749,1.000000,0.925640
2.0,0.961159,0.929901,0.848809,0.925640,1.000000


In [22]:
threshold = 0.95 # lower than 0.95 yields unreasonably big clusters
l = [list(dist_df[dist_df.loc[issue]>threshold].index) for issue in issues_list]

G = to_graph(l)
components = list(connected_components(G))

for aset in components:
    if len(aset)>1:
        for element in aset:
              print(element)
        print("\n")

0.0
1.0
2.0
4.0
5.0
38.0
7.0
44.0
12.0
22.0


8.0
40.0
37.0
46.0


9.0
10.0
19.0


42.0
11.0
56.0
25.0
27.0
62.0


16.0
49.0
91.0


35.0
14.0
15.0
51.0
21.0
55.0
60.0
61.0


17.0
70.0


34.0
52.0
87.0
23.0


26.0
28.0


69.0
29.0


45.0
30.0


57.0
31.0


41.0
66.0
54.0


73.0
43.0
47.0


65.0
67.0




In [23]:
df['cluster_2'] = np.nan

In [24]:
len(components)

107

In [25]:
for i, comp in enumerate(components):
    df['cluster_2'].loc[df['cluster_1'].isin(list(comp))] = i
    print(i, comp)

/opt/conda/envs/Python36/lib/python3.6/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


0 {0.0, 1.0, 2.0, 4.0, 5.0, 38.0, 7.0, 44.0, 12.0, 22.0}
1 {3.0}
2 {6.0}
3 {8.0, 40.0, 37.0, 46.0}
4 {9.0, 10.0, 19.0}
5 {42.0, 11.0, 56.0, 25.0, 27.0, 62.0}
6 {13.0}
7 {16.0, 49.0, 91.0}
8 {35.0, 14.0, 15.0, 51.0, 21.0, 55.0, 60.0, 61.0}
9 {24.0}
10 {17.0, 70.0}
11 {33.0}
12 {18.0}
13 {20.0}
14 {34.0, 52.0, 87.0, 23.0}
15 {26.0, 28.0}
16 {69.0, 29.0}
17 {45.0, 30.0}
18 {57.0, 31.0}
19 {32.0}
20 {36.0}
21 {39.0}
22 {41.0, 66.0, 54.0}
23 {73.0, 43.0, 47.0}
24 {48.0}
25 {50.0}
26 {53.0}
27 {58.0}
28 {59.0}
29 {63.0}
30 {64.0}
31 {65.0, 67.0}
32 {68.0}
33 {71.0}
34 {72.0}
35 {74.0}
36 {75.0}
37 {76.0}
38 {77.0}
39 {78.0}
40 {79.0}
41 {80.0}
42 {81.0}
43 {82.0}
44 {83.0}
45 {84.0}
46 {85.0}
47 {86.0}
48 {88.0}
49 {89.0}
50 {90.0}
51 {92.0}
52 {93.0}
53 {94.0}
54 {95.0}
55 {97.0}
56 {96.0}
57 {98.0}
58 {99.0}
59 {100.0}
60 {101.0}
61 {102.0}
62 {103.0}
63 {104.0}
64 {105.0}
65 {106.0}
66 {107.0}
67 {108.0}
68 {109.0}
69 {110.0}
70 {111.0}
71 {112.0}
72 {113.0}
73 {115.0}
74 {114.0}
75 {116.

In [26]:
issues_list = list(df['cluster_2'].value_counts().index)

# Calculate the mean for the vectors corresponding to each issue
vec_list = [np.mean(np.array(list(compress(vecs_s, list(df[rand]['cluster_2']==issue)))), axis  = 0) for issue in issues_list]

In [27]:
dist = 1-pairwise_distances(np.array(vec_list), metric="cosine")

# Put the distance matrix in a dataframe with index and column names
dist_df = pd.DataFrame(dist, index = issues_list, columns=issues_list)
dist_df.iloc[0:5,0:5]

,0.0,1.0,4.0,2.0,8.0
0.0,1.000000,0.860468,0.849357,0.825983,0.903086
1.0,0.860468,1.000000,0.776079,0.891490,0.772692
4.0,0.849357,0.776079,1.000000,0.755381,0.760120
2.0,0.825983,0.891490,0.755381,1.000000,0.734871
8.0,0.903086,0.772692,0.760120,0.734871,1.000000


In [28]:
threshold = 0.92 
l = [list(dist_df[dist_df.loc[issue]>threshold].index) for issue in issues_list]

G = to_graph(l)
components = list(connected_components(G))

print(len(components))

for aset in components:
    if len(aset)>1:
        for element in aset:
              print(element)
        print("\n")

63
0.0
32.0
33.0
70.0
39.0
8.0
9.0
10.0
12.0
14.0
46.0
16.0
61.0
52.0
21.0
27.0
28.0
29.0


4.0
37.0


19.0
3.0
20.0
6.0


24.0
5.0


15.0
7.0


35.0
38.0
71.0
42.0
13.0
47.0
49.0
23.0


41.0
50.0
22.0
55.0
56.0
58.0
60.0


18.0
26.0


40.0
43.0
48.0
30.0
31.0


25.0
36.0


44.0
53.0


59.0
51.0




In [29]:
df['cluster_3'] = np.nan

In [30]:
for i, comp in enumerate(components):
    df['cluster_3'].loc[df['cluster_2'].isin(list(comp))] = i
    print(i, comp)

/opt/conda/envs/Python36/lib/python3.6/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


0 {0.0, 32.0, 33.0, 70.0, 39.0, 8.0, 9.0, 10.0, 12.0, 14.0, 46.0, 16.0, 61.0, 52.0, 21.0, 27.0, 28.0, 29.0}
1 {1.0}
2 {4.0, 37.0}
3 {2.0}
4 {19.0, 3.0, 20.0, 6.0}
5 {24.0, 5.0}
6 {15.0, 7.0}
7 {17.0}
8 {11.0}
9 {35.0, 38.0, 71.0, 42.0, 13.0, 47.0, 49.0, 23.0}
10 {41.0, 50.0, 22.0, 55.0, 56.0, 58.0, 60.0}
11 {18.0, 26.0}
12 {40.0, 43.0, 48.0, 30.0, 31.0}
13 {25.0, 36.0}
14 {34.0}
15 {44.0, 53.0}
16 {45.0}
17 {59.0, 51.0}
18 {54.0}
19 {57.0}
20 {62.0}
21 {63.0}
22 {64.0}
23 {65.0}
24 {66.0}
25 {67.0}
26 {68.0}
27 {69.0}
28 {72.0}
29 {73.0}
30 {74.0}
31 {75.0}
32 {76.0}
33 {77.0}
34 {78.0}
35 {79.0}
36 {81.0}
37 {80.0}
38 {82.0}
39 {83.0}
40 {84.0}
41 {85.0}
42 {87.0}
43 {86.0}
44 {88.0}
45 {89.0}
46 {90.0}
47 {91.0}
48 {92.0}
49 {94.0}
50 {93.0}
51 {95.0}
52 {96.0}
53 {97.0}
54 {98.0}
55 {99.0}
56 {100.0}
57 {101.0}
58 {102.0}
59 {104.0}
60 {103.0}
61 {105.0}
62 {106.0}


In [36]:
len(df['cluster_3'].unique())

63

## We've reduced the number of clusters from 166 to 63!

In [32]:
project.save_data("df_clustered.csv", df.to_csv())

{'file_name': 'df_clustered.csv',
 'message': 'File saved to project storage.',
 'bucket_name': 'qmssdseprojectfall2019-donotdelete-pr-ju1hanupbqko6m',
 'asset_id': 'b24b5117-c47f-4d94-ae30-125dae62d3fa'}